In [1]:
import tensorflow as tf
import tf2onnx
import onnx
import numpy as np

print("TF:", tf.__version__)        # Should be 2.16.1
print("Protobuf:", tf2onnx.__version__)  # Should load without error
print("NumPy:", np.__version__)     # Should be < 2.0
print("✅ All good!")



TF: 2.16.1
Protobuf: 1.16.1
NumPy: 1.26.4
✅ All good!


In [2]:
import os
import tensorflow as tf
import tf2onnx
import onnx

# Paths
MODEL_DIR = r"C:\Users\leopo\OneDrive\Bureau\UNITY\models"
keras_path = os.path.join(MODEL_DIR, "malaria_cnn_final.keras")
onnx_path = os.path.join(MODEL_DIR, "malaria_cnn_final.onnx")

# Load your trained model
model = tf.keras.models.load_model(keras_path)
print("✅ Model loaded")

# Define input signature (must match training)
spec = (tf.TensorSpec((None, 128, 128, 3), tf.float32, name="input"),)

# Convert to ONNX
print("🔄 Converting to ONNX...")
onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=13  # Unity Barracuda compatible
)

# Save ONNX file
onnx.save(onnx_model, onnx_path)
print(f"✅ ONNX model saved to: {onnx_path}")

✅ Model loaded
🔄 Converting to ONNX...


AttributeError: 'Sequential' object has no attribute 'output_names'

In [3]:
import tensorflow as tf
import os

MODEL_DIR = r"C:\Users\leopo\OneDrive\Bureau\UNITY\models"
keras_path = os.path.join(MODEL_DIR, "malaria_cnn_final.keras")

# Load original model
original_model = tf.keras.models.load_model(keras_path)

In [4]:
# Create input layer
input_layer = tf.keras.Input(shape=(128, 128, 3))

# Pass through original model layers
x = input_layer
for layer in original_model.layers:
    x = layer(x)

# Create new functional model
functional_model = tf.keras.Model(inputs=input_layer, outputs=x)

print("✅ Model converted to functional style")

✅ Model converted to functional style


In [5]:
import tf2onnx
import onnx

# Define input signature
spec = (tf.TensorSpec((None, 128, 128, 3), tf.float32, name="input"),)

# Convert
onnx_model, _ = tf2onnx.convert.from_keras(
    functional_model,
    input_signature=spec,
    opset=13
)

# Save
onnx_path = os.path.join(MODEL_DIR, "malaria_cnn_final.onnx")
onnx.save(onnx_model, onnx_path)
print(f"✅ ONNX model saved to: {onnx_path}")

✅ ONNX model saved to: C:\Users\leopo\OneDrive\Bureau\UNITY\models\malaria_cnn_final.onnx


In [8]:
data_dir = r"C:\Users\leopo\OneDrive\Bureau\UNITY\cell_images"

# Load datasets
raw_train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.3,
    subset="training",
    seed=42,
    shuffle=True
)

raw_val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.3,
    subset="validation",
    seed=42,
    shuffle=True
)

# ✅ Get class names BEFORE transformation
class_names = raw_train_ds.class_names
print("✅ Class names:", class_names)

# Normalize
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = raw_train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = raw_val_ds.map(lambda x, y: (normalization_layer(x), y))

# Prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

Found 27558 files belonging to 2 classes.
Using 19291 files for training.
Found 27558 files belonging to 2 classes.
Using 8267 files for validation.
✅ Class names: ['0_Uninfected', '1_Parasitized']


In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 93,377 (364.75 KB)

 Trainable params: 93,377 (364.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
  9/603 ━━━━━━━━━━━━━━━━━━━━ 11:23 1s/step - accuracy: 0.4253 - loss: 0.7137  

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

In [1]:
data_dir = r"C:\Users\leopo\OneDrive\Bureau\UNITY\cell_images"

# Load datasets
raw_train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.3,
    subset="training",
    seed=42,
    shuffle=True
)

raw_val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.3,
    subset="validation",
    seed=42,
    shuffle=True
)

# ✅ Get class names BEFORE transformation
class_names = raw_train_ds.class_names
print("✅ Class names:", class_names)

# Normalize
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = raw_train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = raw_val_ds.map(lambda x, y: (normalization_layer(x), y))

# Prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

NameError: name 'tf' is not defined

In [2]:
import tensorflow as tf  # ← ADD THIS LINE

data_dir = r"C:\Users\leopo\OneDrive\Bureau\UNITY\cell_images"

# Load datasets
raw_train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.3,
    subset="training",
    seed=42,
    shuffle=True
)

raw_val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.3,
    subset="validation",
    seed=42,
    shuffle=True
)

# ✅ Get class names BEFORE transformation
class_names = raw_train_ds.class_names
print("✅ Class names:", class_names)

# Normalize
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = raw_train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = raw_val_ds.map(lambda x, y: (normalization_layer(x), y))

# Prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

Found 27558 files belonging to 2 classes.
Using 19291 files for training.
Found 27558 files belonging to 2 classes.
Using 8267 files for validation.
✅ Class names: ['0_Uninfected', '1_Parasitized']


In [3]:
import tensorflow as tf

# Build model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary output
])

# Compile
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 93,377 (364.75 KB)

 Trainable params: 93,377 (364.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train for 10 epochs
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1
)

Epoch 1/10
188/603 ━━━━━━━━━━━━━━━━━━━━ 1:10:07 10s/step - accuracy: 0.8614 - loss: 0.3891

In [ ]:
# Train for 10 epochs
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1
)